In [1]:
import numpy as np
import matplotlib.pyplot as plt

# TODO: make 1D truss linear solver with fixed area and constant material properties


In [55]:
class TrussElement:
    def __init__(self, node_indeces, E, A, length, q,shape_function="linear"):
        self.nodes = node_indeces
        self.E = E
        self.A = A
        self.length = length
        self.q = q
        self.shape_function = shape_function

    def stiffness_matrix(self):
        if self.shape_function == "linear":
            k = self.E * self.A / self.length
            return k*np.array([[1, -1], [-1, 1]])
        elif self.shape_function == "quadratic":
            k = self.E * self.A / (self.length*3)
            return k*np.array([[7, -4, 1], [-4, 8, -4], [1, -4, 7]])

    def load_vector(self):
        # Calculate the load vector for the element
        if self.shape_function == "linear":
            return np.array([self.q*self.length / 2, self.q*self.length / 2])
        elif self.shape_function == "quadratic":
            return np.array([self.q*self.length / 6, 4*self.q*self.length / 6, self.q*self.length / 6])

    def calculate_node_coordinates(self):
        # Calculate the coordinates of the nodes
        pass

    def __repr__(self):
        return f"TrussElement(nodes={self.nodes}"


In [56]:
# N, mm, MPa

class Truss1D:
    def __init__(self, num_elem, E, A, L, q = 0, F_applied_at_end = 0, shape_function="linear"):
        self.num_elem = num_elem
        self.E = E
        self.A = A
        self.L = L
        self.q = q # distributed load, N/mm
        self.F_applied_at_end = F_applied_at_end # N
        self.shape_function = shape_function

        self.elements = np.zeros(num_elem, dtype=TrussElement)
        self.num_nodes = self.num_elem + 1 if shape_function == "linear" else 2*self.num_elem + 1

    def create_elements(self):
        # create TrussElemets for a specified length of truss with num_elements
        element_length = self.L / self.num_elem
        for i, element_i in enumerate(range(self.num_elem)):
            if self.shape_function == "linear":
                x1 = element_i * element_length
                x2 = (element_i + 1) * element_length
                node_indeces = [element_i, element_i + 1]
                self.elements[i] = TrussElement(node_indeces, self.E,  self.A, element_length,q=self.q, shape_function=self.shape_function)

            elif self.shape_function == "quadratic":
                nodes = [2*element_i-1, 2*element_i, 2*element_i+1]

    def __repr__(self):
        return self.elements


In [60]:
E=50*10**3
A=75
L=1000

num_elem = 2
my_truss = Truss1D(num_elem, E, A, L, q = 0, F_applied_at_end=10)
my_truss.create_elements()

solver = TrussSolver(my_truss)
solver.assemble_global_matrices()

(array([[ 7500., -7500.,     0.],
        [-7500., 15000., -7500.],
        [    0., -7500.,  7500.]]),
 array([ 0.,  0., 10.]))

In [58]:
class TrussSolver:
    def __init__(self, truss : Truss1D):
        self.truss = truss

        self.K_global = None
        self.F_global = None
        self.displacements = None

    def assemble_global_matrices(self):
        # Assemble the global stiffness and load vector
        self.K_global = np.zeros((self.truss.num_nodes, self.truss.num_nodes))
        for i, element in enumerate(self.truss.elements):
            k_element = element.stiffness_matrix()
            for j in range(2):
                for k in range(2):
                    self.K_global[element.nodes[j], element.nodes[k]] += k_element[j, k]

        self.F_global = np.zeros(self.truss.num_nodes)
        for i, element in enumerate(self.truss.elements):
            for j in range(2):
                f_element = element.load_vector()
                self.F_global[element.nodes[j]] += f_element[j]
        self.F_global[-1] += self.truss.F_applied_at_end

        return self.K_global, self.F_global


    def solve_displacements(self):
        # Solve for displacements
        pass

    def compute_stresses(self):
        # Calculate stresses in each element
        pass


In [6]:

class Plotter:
    def __init__(self, truss_solver):
        self.truss_solver = truss_solver

    def plot_displacements(self):
        # Code to plot displacement vs. analytical solution
        pass

    def plot_stresses(self):
        # Code to plot stress vs. analytical solution
        pass
